In [1]:
import os
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [8]:
data_path="dataset2/rtdata.txt"
all_data = []
with open(data_path, "r", encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        line = [float(x) for x in line.strip().split()]
        all_data.append(line)
all_data[0]

[0.0, 0.0, 0.0, 4.18]

In [9]:
# make dataset
train_data, dev_data = train_test_split(all_data, test_size=10000)
train_data_5, res_data = train_test_split(train_data, train_size=0.05)
# train_data_10, res_data = train_test_split(train_data, train_size=0.1)
# train_data_15, res_data = train_test_split(train_data, train_size=0.15)
# train_data_20, res_data = train_test_split(train_data, train_size=0.20)
# train_data_25, res_data = train_test_split(train_data, train_size=0.25)

/home/LAB/chenty/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
# Evaluation Method
import math
def MAE(y_true, y_pred):
    res = y_true - y_pred
    if res < 0:
        res = - res
    if res > 0 and y_pred > 0:
        res = res*1.00000 / y_pred
        
    return res


def RMSE(y_true, y_pred):
    res = (y_true - y_pred)**2
    if y_pred > 0:
        res = res*1.00000 /y_pred
    else:
        res = res
    return res

def MSE(y_true, y_pred):
    res = (y_true - y_pred)**2
    return res

In [11]:
# UMean
Users = {}
all_scores = []
for line in tqdm(train_data_5):
    user_id = line[0]
    rt  = line[-1]
    if user_id in Users:
        Users[user_id].append(rt)
    else:
        Users[user_id] = [rt]
    all_scores.append(rt)

search_table = {}
for k in Users.keys():
    search_table[k] = sum(Users[k])*1.000 / len(Users[k])

avg_score = sum(all_scores)*1.000 / len(all_scores)

## Evaluate
avg_mse = 0

for line in tqdm(dev_data):
    user_id = line[0]
    y_true = line[-1]
    if user_id in search_table:
        y_pred = search_table[user_id]
    else:
        y_pred = avg_score
    mse_score = MAE(y_true, y_pred)
    avg_mse += mse_score

avg_mse = avg_mse / len(dev_data)
avg_mse

1.3192837409691838

In [12]:
# IMean
Items = {}
all_scores = []
for line in tqdm(train_data_5):
    item_id = line[1]
    rt = line[-1]
    if item_id in Items:
        Items[item_id].append(rt)
    else:
        Items[item_id] = [rt]
    all_scores.append(rt)

search_table = {}

for k in Items.keys():
    search_table[k] = sum(Items[k])*1.000 / len(Items[k])
avg_score = sum(all_scores)*1.000 / len(all_scores)

## Evaluate
avg_mse = 0
for line in tqdm(dev_data):
    item_id = line[1]
    y_true = line[-1]
    if item_id in search_table:
        y_pred = search_table[item_id]
    else:
        y_pred = avg_score
    mse_score = MAE(y_true, y_pred)
    avg_mse += mse_score

avg_mse = avg_mse / len(dev_data)
avg_mse

1.041187998949346

In [13]:
# User based FC

import numpy as np
from scipy.spatial import distance

top_k = 5
Users = np.zeros((142, 4500))
for line in tqdm(train_data_5):
    user_id = int(line[0])
    item_id = int(line[1])
    val = line[-1]
    Users[user_id][item_id] = val

# Evaluating
avg_mse = 0.0
all_true = []
all_pred = []
for line in tqdm(dev_data):
    user_id = int(line[0])
    item_id = int(line[1])
    y_true = line[-1]
    user_vec =Users[user_id]
    all_dis = [distance.cosine(user_vec, x) for x in Users]
    top_k_vec = np.argsort(np.array(all_dis))[1:6]
    all_true.append(y_true)
    ref_item = [Users[x][item_id] for x in top_k_vec]
    y_pred = sum(ref_item) /  len(ref_item)
    all_pred.append(y_pred)

for t, p in zip(all_true, all_pred):
    avg_mse += MAE(y_true, y_pred)

avg_mse = avg_mse /len(all_true)
avg_mse


0.32926829268296887

In [ ]:
# Item based FC

import numpy as np
from scipy.spatial import distance
from multiprocessing import Pool

top_k = 5
Items = np.zeros((4500, 142))
for line in tqdm(train_data_5):
    user_id = int(line[0])
    item_id = int(line[1])
    val = line[-1]
    Items[item_id][user_id] = val

# Evaluating
avg_mse = 0.0
all_true = []
all_pred = []
for line in tqdm(dev_data):
    user_id = int(line[0])
    item_id = int(line[1])
    y_true = line[-1]
    item_vec = Items[item_id]
#     def f(y):
#         return distance.cosine(item_vec, y)
#     with Pool(64) as p:
#         all_dis = p.map(f, Items)
    all_dis = [distance.cosine(item_vec, x) for x in Items] 
    top_k_vec = np.argsort(np.array(all_dis))[1:6]
    all_true.append(y_true)
    ref_User = [Items[x][user_id] for x in top_k_vec]
    y_pred = sum(ref_item) /  len(ref_item)
    all_pred.append(y_pred)

for t, p in zip(all_true, all_pred):
    avg_mse += MAE(y_true, y_pred)

avg_mse = avg_mse /len(all_true)
avg_mse

In [6]:
# Hybrid CF

import numpy as np
from scipy.spatial import distance
from multiprocessing import Pool

top_k = 5
Items = np.zeros((4500, 142))
Users = np.zeros((142, 4500))
for line in tqdm(train_data_5):
    user_id = int(line[0])
    item_id = int(line[1])
    val = line[-1]
    Items[item_id][user_id] = val
    Users[user_id][item_id] = val

# Evaluating
avg_mse = 0.0
all_true = []
all_pred = []
for line in tqdm(dev_data):
    user_id = int(line[0])
    item_id = int(line[1])
    y_true = line[-1]
    item_vec = Items[item_id]
    user_vec = Users[user_id]
    # get similarity from Items
    all_dis = [distance.cosine(item_vec, x) for x in Items] 
    top_k_vec = np.argsort(np.array(all_dis))[1:6]
    all_true.append(y_true)
    ref_User = [Items[x][user_id] for x in top_k_vec]
    # get similarity from Users
    user_vec =Users[user_id]
    all_dis = [distance.cosine(user_vec, x) for x in Users]
    top_k_vec = np.argsort(np.array(all_dis))[1:6]
    ref_item = [Users[x][item_id] for x in top_k_vec]
    
    ref_vals = ref_User + ref_item
    y_pred = sum(ref_vals) / len(ref_vals)
    all_pred.append(y_pred)

for t, p in zip(all_true, all_pred):
    avg_mse += MAE(y_true, y_pred)

avg_mse = avg_mse /len(all_true)
avg_mse